In [1]:
import numpy as np
import pandas as pd

In [2]:
%%time
stock_file_name = 'data/sp500_3yrs.csv'
df = pd.read_csv(stock_file_name, index_col=0, parse_dates=True)

CPU times: user 25 ms, sys: 3.89 ms, total: 28.9 ms
Wall time: 32.8 ms


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 754 entries, 2022-10-17 to 2025-10-17
Columns: 503 entries, A to ZTS
dtypes: float64(503)
memory usage: 2.9 MB


In [4]:
stocks = df.columns.to_list()
stocks = [s.replace('-', '_') for s in stocks]
df.columns = stocks

In [5]:
stocks_with_missing_data = df.columns[df.isna().any()].tolist()
stocks_with_missing_data

['GEHC', 'GEV', 'KVUE', 'SOLV', 'VLTO']

In [6]:
for col in stocks_with_missing_data:
    missing_dates = df[df[col].isna()].index
    print('{} is missing from {} to {}.'.format(col, missing_dates[0].date(), missing_dates[-1].date()))

GEHC is missing from 2022-10-17 to 2022-12-14.
GEV is missing from 2022-10-17 to 2024-03-26.
KVUE is missing from 2022-10-17 to 2023-05-03.
SOLV is missing from 2022-10-17 to 2024-03-25.
VLTO is missing from 2022-10-17 to 2023-10-03.


Note that some stocks will have NaNs for early dates if they weren’t yet public.  

Models working with such datasets have two options:

1. Eliminate these columns entirely if the time period is critical for the analysis, ignoring stocks with short time series.

2. Use only the data from when the stocks were listed, so that there are no empty columns in the dataset.

However, since the latest missing data among these stocks extends up to 2024-03-26, option #2 would discard a large portion of the data. Therefore, I decided to drop these stocks entirely from the dataset.

In [7]:
df.drop(columns=stocks_with_missing_data, inplace=True)

In [14]:
if not df.isna().any().any():
    print('No NaNs remaining')
    print('Number of stocks in S&P 500: {}'.format(len(df.columns)))

No NaNs remaining
Number of stocks in S&P 500: 498


In [9]:
df.to_csv('data/clean_sp500_3yrs.csv')